In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [2]:
class FashionMNISTtask1(datasets.FashionMNIST):
  def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
    super(FashionMNISTtask1, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
    self.classes = self.classes[:6]

  def __getitem__(self, index):
    img, target = super(FashionMNISTtask1, self).__getitem__(index)
    if target < 6:
        return img, target
    else:
        return img, -1

In [3]:
class FashionMNISTtask2(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(FashionMNISTtask2, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
        self.classes = self.classes[6:]

    def __getitem__(self, index):
        img, target = super(FashionMNISTtask2, self).__getitem__(index)
        if target >= 6:
            return img, target
        else:
            return img, -1

In [ ]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset_1 = FashionMNISTtask1(root='./data1', train=True, transform=transform, download=True)
test_dataset_1 = FashionMNISTtask1(root='./data1', train=False, transform=transform, download=True)

In [ ]:
train_dataset_2 = FashionMNISTtask2(root='./data2', train=True, transform=transform, download=True)
test_dataset_2 = FashionMNISTtask2(root='./data2', train=False, transform=transform, download=True)
test_dataset_3 = datasets.FashionMNIST(
    root='data3',
    train=False,
    download=True,
    transform=transform
)

In [6]:
train_dataset_filtered_old = [data for data in train_dataset_1 if data[1] != -1]
test_dataset_filtered_old = [data for data in test_dataset_1 if data[1] != -1]

train_dataset_filtered_new = [data for data in train_dataset_2 if data[1] != -1]
test_dataset_filtered_new = [data for data in test_dataset_2 if data[1] != -1]

In [7]:
train_dataloader_old = DataLoader(train_dataset_filtered_old, batch_size=64, shuffle=True)
test_dataloader_old = DataLoader(test_dataset_filtered_old, batch_size=256, shuffle=False)

for X, y in train_dataloader_old:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [8]:
train_dataloader_new = DataLoader(train_dataset_filtered_new, batch_size=64, shuffle=True)
test_dataloader_new = DataLoader(test_dataset_filtered_new, batch_size=256, shuffle=False)

In [9]:
eval_dataloader = DataLoader(test_dataset_3, batch_size=256, shuffle=True)

In [10]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device.")

Using cuda device.


In [11]:
class CNN(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32*14*14, hidden_size)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        logits = self.classifier(x)

        return logits

In [12]:
def train(model, dataloader, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")


In [13]:
def test(model, dataloader, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test loss:\nAccuracy: {100*correct:>0.1f}, Avg Loss: {test_loss:>8f}\n")

In [14]:
def val(model, epoch):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(eval_dataloader):
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [15]:
# Task 1: Train the base model
base_model = CNN()
base_model = base_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    print(f"Task 1 - Epoch {epoch+1}:---------------------")
    train(base_model, train_dataloader_old, loss_fn, optimizer)
    test(base_model, test_dataloader_old, loss_fn)

print("Done!")

# Save the state of the trained base model
torch.save(base_model.state_dict(), "base_model_task1.pth")

# Task 2: Meta Continual Learning
meta_model = CNN()
meta_model = meta_model.to(device)
meta_optimizer = torch.optim.Adam(meta_model.parameters(), lr=0.001)

# Load weights from the trained base model (Task 1)
meta_model.load_state_dict(torch.load("base_model_task1.pth"))

for epoch in range(num_epochs):
    print(f"Task 2 - Epoch {epoch+1}:---------------------")
    train(meta_model, train_dataloader_new, loss_fn, meta_optimizer)
    test(meta_model, test_dataloader_new, loss_fn)
    val(meta_model, epoch)

Task 1 - Epoch 1:---------------------
Loss: 2.554302,    64/36000
Loss: 0.448380,  6464/36000
Loss: 0.431891, 12864/36000
Loss: 0.237091, 19264/36000
Loss: 0.203146, 25664/36000
Loss: 0.117528, 32064/36000
Test loss:
Accuracy: 93.3, Avg Loss: 0.185902

Task 1 - Epoch 2:---------------------
Loss: 0.127286,    64/36000
Loss: 0.125016,  6464/36000
Loss: 0.370917, 12864/36000
Loss: 0.224994, 19264/36000
Loss: 0.095824, 25664/36000
Loss: 0.163408, 32064/36000
Test loss:
Accuracy: 92.9, Avg Loss: 0.189447

Task 1 - Epoch 3:---------------------
Loss: 0.128673,    64/36000
Loss: 0.110751,  6464/36000
Loss: 0.090187, 12864/36000
Loss: 0.122689, 19264/36000
Loss: 0.172416, 25664/36000
Loss: 0.142051, 32064/36000
Test loss:
Accuracy: 93.7, Avg Loss: 0.181664

Task 1 - Epoch 4:---------------------
Loss: 0.157002,    64/36000
Loss: 0.126877,  6464/36000
Loss: 0.029186, 12864/36000
Loss: 0.025264, 19264/36000
Loss: 0.043099, 25664/36000
Loss: 0.077168, 32064/36000
Test loss:
Accuracy: 94.2, Avg 